In [ ]:
1 + 1

In [ ]:
using InteractiveUtils, DrWatson, Comonicon
if isdefined(Main, :IJulia) && Main.IJulia.inited
    using Revise
else
    ENV["GKSwstype"] = 100 # suppress warnings during gif saving
end
versioninfo()
@quickactivate

In [ ]:
using Plots, ProgressMeter, Logging
theme(:bright; size=(300, 300))

In [ ]:
using Random, Turing, BayesianSymbolic
using ExprOptimization.ExprRules
using PyCall
pd = pyimport("pandas")

includef(args...) = isdefined(Main, :Revise) ? includet(args...) : include(args...)
includef(srcdir("utility.jl"))
includef(srcdir("app_inf.jl"))
includef(srcdir("sym_reg.jl"))
includef(srcdir("network.jl"))
includef(srcdir("exp_max.jl"))
includef(srcdir("analyse.jl"))
includef(srcdir("dataset.jl"))
# Suppress warnings of using _varinfo
with_logger(SimpleLogger(stderr, Logging.Error)) do
    includef(srcdir("scenarios", "nbody.jl"))
    includef(srcdir("scenarios", "bounce.jl"))
    includef(srcdir("scenarios", "mat.jl"))
end

In [ ]:
function report_synth(dataset; rg_seed=1:5, rg_shuffleseed=1:5, namewithweak=false)
    dataset = "synth/$dataset"
    df = pd.DataFrame(
        columns=("Method", "Normliased RMSE (train)", "Normliased RMSE (test)", "EM seed", "Data seed")
    )
    forces = Dict{Int, Any}()

    count = 1
    @showprogress for seed in rg_seed, shuffleseed in rg_shuffleseed
        hps = (ntrains=5, seed=seed, shuffleseed=shuffleseed)
        if namewithweak
            hps = (hps..., weak=false)
        end

        dir = datadir(dataset)
        scenarios, attributes = loaddata(dir, hps.ntrains; shuffleseed=hps.shuffleseed)
        scenarios_test, attributes_test = loaddata(dir, 20; shuffleseed=hps.shuffleseed, istrain=false)

        local res
        try
            res = wload(resultsdir(dataset, savename(hps; connector="-"), "em.jld2"))
        catch
            println("seed $(hps.seed) unfinished")
        end

        @unpack ScenarioModel, latentname, ealg, malg, elike, mlike, trace = res

        for (force, method) in [(trace[end].force, "BSP"), (ZeroForce(), "Zero")]
            if method == "BSP"
                forceexpr = BayesianSymbolic.get_executable(force.tree, force.grammar)
                @info "Count $count" forceexpr
                # Track expressions
                forces[count] = (force.constant, forceexpr)
            end
            evalres = evalforce(ScenarioModel, scenarios, attributes, force, ealg, elike, mlike)
            evalres_test = evalforce(ScenarioModel, scenarios_test, attributes_test, force, ealg, elike, mlike)

            # Add to data frame
            df.loc[count] = [method, evalres.normrmse, evalres_test.normrmse, seed, shuffleseed]
            count = count + 1
        end
    end
    
    return df, forces
end

function savedf(df, dataset)
    df.groupby(["Method"]).get_group("BSP").to_csv("pretty_paper/raw/em-$dataset-bsp.csv", columns=["Normliased RMSE (test)", "EM seed", "Data seed"], header=false)
    df.groupby(["Method"]).get_group("Zero").to_csv("pretty_paper/raw/em-$dataset-zero.csv", columns=["Normliased RMSE (test)", "EM seed", "Data seed"], header=false)
end

NBODY

In [ ]:
df, forces = report_synth("nbody")
df

In [ ]:
savedf(df, "nbody")

In [ ]:
df.groupby(["Method"]).median()

In [ ]:
df.groupby(["Method"]).quantile([.25, .75])

BOUNCE

In [ ]:
df, forces = report_synth("bounce")
df

In [ ]:
df.groupby(["Method"]).median()

In [ ]:
df.groupby(["Method"]).quantile([.25, .75])

MAT

In [ ]:
df, forces = report_synth("mat"; rg_seed=0:9, rg_shuffleseed=-1:1, namewithweak=true)
df

In [ ]:
df.groupby(["Method"]).median()

In [ ]:
df.groupby(["Method"]).quantile([.25, .75])

In [ ]:
df.groupby(["Method"]).mean()

In [ ]:
df.groupby(["Method"]).std()

In [ ]:
df.groupby(["Method"]).min()

In [ ]:
df.groupby(["Method"]).max()